In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

! pip install transformers==4.27.4

! pip install torchtext==0.10.1

import torch
device = torch.device("cuda")
torch.cuda.init()

from google.colab import drive
drive.mount('/content/gdrive')

from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, TextDataset

In [ ]:
model_path = "/content/gdrive/My Drive/model/chitchat_generator.pt"

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', padding_side='left')
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.load_state_dict(torch.load(model_path, map_location=device))

In [ ]:
test_path = '/content/gdrive/My Drive/dataset/combined_test.txt'

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
import re
def getResponse(input_text, model,tokenizer, device):
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  if(len(input_ids)<=0):
    print(input_text)
    return None
  input_ids = input_ids.to(device)
  model = model.to(device)
  output_ids = model.generate(input_ids,pad_token_id=tokenizer.eos_token_id, max_length=70,early_stopping=True)
  output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  messages = output_text.split("\n")
  first_bot_response = None
  for message in messages:
    if message.startswith("Bot:"):
        first_bot_response = message.strip()
        break
  return first_bot_response

In [ ]:
print(getResponse("User: Hello, I am feeling very sad",model, tokenizer, device))

In [ ]:
with open('/content/gdrive/My Drive/dataset/combined_test.txt', 'r', encoding='utf-8') as f:
    test_data = f.readlines()

In [ ]:
len(test_data)

In [ ]:
def prepare(test_data):
  input = {}
  response = {}
  conv_id = 1
  idx = 1
  for i in range(len(test_data)):
    input[idx]=""
    response[idx]=""
    if(test_data[i]=="\n" or test_data[i]=='\n'):
      idx+=1
  for data in test_data:
    if(conv_id>=idx):
      break
    if(data.startswith("User")):
      input[conv_id]+= data 
    elif(data.startswith("Bot")):
      response[conv_id]+=data
    else:
      conv_id+=1
  return input,response

In [ ]:
input, response = prepare(test_data)

In [ ]:
# from tqdm import tqdm
# def getResponses(input, response, model,tokenizer, device):
#   generated_response = {}
#   with open("/content/gdrive/My Drive/dataset/generated_responses.json", 'a') as f:
#     for idx, text in tqdm(input.items()):
#       value = getResponse(text, model,tokenizer, device)
#       if(value==None):
#         response2[idx]="None"
#       generated_response[idx] = value
#       f.write(str(idx)+ ":" )
#       f.write(str(value))
#       f.write("\n")
#   return generated_response

In [ ]:
from tqdm import tqdm
def getResponses(input, response, model,tokenizer, device):
  generated_response = {}
  for idx, text in tqdm(input.items()):
    value = getResponse(text, model,tokenizer, device)
    if(value==None):
      response2[idx]="None"
    generated_response[idx] = value
  return generated_response

In [ ]:
input_demo_keys = list(input.keys())[:7000]
input_demo = {key: input[key] for key in input_demo_keys}

In [ ]:
response_demo_keys = list(response.keys())[:7000]
response_demo = {key: response[key] for key in response_demo_keys}
response2=response_demo.copy()

In [ ]:
generated_response = getResponses(input_demo, response_demo, model,tokenizer, device)

In [ ]:
import json
# open file for writing
with open("/content/gdrive/My Drive/dataset/generated_responses.json", "w") as outfile:
    # write dictionary to file in JSON format
    json.dump(generated_response, outfile)

In [ ]:
import json
# open file for writing
with open("/content/gdrive/My Drive/dataset/modified_responses.json", "w") as outfile:
    # write dictionary to file in JSON format
    json.dump(response2, outfile)

In [ ]:
with open("/content/gdrive/My Drive/dataset/generated_responses2.json", "r") as rf:
  data = rf.readlines()
len(data)

In [ ]:
formatted_response = []
for res in data:
  res.strip()
  if("Bot" in res):
    res = res[res.find("Bot"):len(res)-1]
  formatted_response.append(res)

In [ ]:
! pip install bert-score


In [ ]:
from bert_score import score
P, R, F1 = score(formatted_response, list(response.values()), lang="en", verbose=True)

In [ ]:
print(P.mean().item())
print(R.mean().item())
print(F1.mean().item())

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
bleu_score = corpus_bleu(formatted_response, list(response.values()))
bleu_score

In [ ]:
! pip install rouge

In [ ]:
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(formatted_response, list(response.values()), avg=True)